# Resumo do código

>### Entra: 1_Ninjas e 2_Delta  
>### Sai: OUT_DataFusion_Wide e OUT_DataFusion_Wide_2022

---
O objectivo é receber dados dos Ninjas e da Delta e devolver um ficheiro Wide que permita verificar o comportamento de vários produtos em diversas lojas através de representações gráficas. 

Devolve um ficheiro que entra no R para gerar o Dashboard.

---
- Inputs

> __Dados dos ninjas__ em formato Wide
> - 1's e 0's consoante a presença e ausência do produto, Data e Loja

> __Dados da Delta__ em formato Wide
> - Stocks e trânsito, Sellout dia anterior

- Outputs

> __Ficheiro Wide__ 

> __Métricas novas:__
> - Roturas de Stock, Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - STK
> - MSA
> - Balanço
> - Dias para a rotura de stock e de prateleira


In [1]:
import pandas as pd
import numpy as np

def escrever_excel(dfa, nome):
    dfa.to_excel('D:\\B&N Dados\\Delta\\Padrão\\Padrão_xx_2023\\%s.xlsx' %nome, index=False)

# Ler e Mudar nomes

- # `Ninjas (sinal)`

In [ ]:
#Info Ninja
dfNinjas=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão\\Padrão_xx_2023\\1Ninjas_Limpo.xlsx")

> Definir variáveis importantes
> - Se houver um ficheiro com os produtos

In [4]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão\\Padrão_xx_2023\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
resto = dfNinjas.columns.difference(produtos)

> - Se se quiser usar o primeiro e o último produtos

> - Reposição

In [7]:
# Ler ficheiro
dfRepos=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão\\Padrão_05_2023\\LojasReposição.xlsx")

# Criar coluna de reposição
dfNinjas['Reposição'] = [1 if val in dfRepos['Loja'].values else 0 for val in dfNinjas['Loja']]

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\B&N Dados\\Delta\\Padrão_05_2023\\LojasReposição.xlsx'

> - Id

In [ ]:
# Extract the day and month components from the 'datetime' column
dfNinjas['day'] = pd.to_datetime(dfNinjas['DATA']).dt.day
dfNinjas['month'] = pd.to_datetime(dfNinjas['DATA']).dt.month

# Create a new column combining day and month
dfNinjas['day_month'] = dfNinjas['day'].astype(str) + '-' + dfNinjas['month'].astype(str)

# Assign unique IDs to each unique day-month combination
dfNinjas['id'], _ = pd.factorize(dfNinjas['day_month'])

dfNinjas = dfNinjas.drop(columns=['day', 'month', 'day_month'])

> - Número de produtos presentes

In [5]:
# Criar coluna com o número dos produtos em causa presentes no instante

dfNinjas["Num_Produtos"] = dfNinjas[produtos].sum(axis=1)

dfNinjas.loc[(dfNinjas["Num_Produtos"] == 0) & (dfNinjas[produtos[1]].isna()), "Num_Produtos"] = np.nan

> - Vendedor

In [ ]:
# Ler ficheiro
dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\Vendedor.xlsx", sheet_name = "Lista Lojas Sonae")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfNinjas = pd.merge(dfNinjas, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

---

- #  `Delta`

In [16]:
# Ler o ficheiro long com Stocks e Fornecimento

dfDelta = pd.read_excel("D:\\B&N Dados\\Delta\\Padrão\\Padrão_xx_2023\\2_Delta_Limpo.xlsx", sheet_name="Sheet1")

# <font color=green>Mergir</font>

In [19]:
# Dataframe mergido no fim
dfMeio=dfNinjas.copy()


for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta[dfDelta["DESC_ARTIGO"]==coluna][["DATA", "STORE", "STOCK", "PRES_STOCK", "INTRANSIT", "EXPECTED", "SELLOUT"]]   #Dados Ninjas, seleccionar colunas do café específico
 
    #Juntar dados de acordo com o dia e a loja
    dfMeio=pd.merge(dfMeio, dfStocks, how="left", on = ["DATA","STORE"])  
    
    #Mudar nomes para ser adaptado a cada produto
    dfMeio=dfMeio.rename(columns={"STOCK": "STOCK %s" % coluna, "PRES_STOCK":"PRESLINEAR %s" % coluna, "INTRANSIT":"INTRANSIT %s" % coluna, "EXPECTED":"EXPECTED %s" % coluna, "SELLOUT":"SELLOUT %s" % coluna})                       #Nomear coluna nova
    
    

### Base que vai ser trabalhada

In [21]:
dfFinal=dfMeio.copy()

### <font color=green>Fim</font>

---

# Já temos DataFrame com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected


## Fazer:

- STK 
- Balanço
- Rotura 
- Pré-rotura
- Sinal
- Ciclos
- Adequação de Stock
- Dias para a rotura de stock
- Dias para a rotura de prateleira

---

> STK

In [22]:
for i in produtos:

    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    
    stk = "STK %s" %i
      
    dfFinal[stk] = dfFinal[stock] + dfFinal[esperado] + dfFinal[transito]

> Balanço

In [ ]:
for i in produtos:
    stk = "STK %s" %i
    #msa = "MSA %s" %i
    #sellout = "SELLOUT %s" %i
    
    balance = "BALANCE %s" %i
    
    dfFinal[balance] = dfFinal[msa] / dfFinal[stk]
    
    

> Rotura

In [23]:
for i in produtos:
    rotura  = "ROTURA %s" % i
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    
    dfFinal[rotura] = np.where((dfFinal[stock] > 0) & ~(dfFinal[i].isna()), 0, 
                      np.where((dfFinal[stock] <= 0) & (dfFinal[preslinear] > 0) & ~(dfFinal[i].isna()), 1, 
                      np.nan))


> Pré-Rotura

In [24]:
for i in produtos:
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    pre_rotura= "PRÉ-ROTURA %s" % i
    
    dfFinal[pre_rotura] = (dfFinal[stock] < dfFinal[preslinear]).astype(int)

>Sinal

In [25]:
#dfFinal["Sinal"]= pd.Series(dtype='int')

# Define the mappings of numbers to strings
mapping = {1: "Presente com Stock",
           2: "Ausente com Stock",
           3: "Presente sem Stock",
           4: "Ausente sem Stock",
           5: "Presente sem Registo",
           6: "Ausente sem Registo"}

for i in produtos:
    sinal = "SINAL %s" % i
    stock  = "STOCK %s" % i
    
    dfFinal[sinal] = np.where((dfFinal[stock] > 0) & (dfFinal[i] == 1), 1,
                     np.where((dfFinal[stock] > 0) & (dfFinal[i] == 0), 2,
                     np.where((dfFinal[stock] <= 0) & (dfFinal[i] == 1), 3,
                     np.where((dfFinal[stock] <= 0) & (dfFinal[i] == 0), 4,
                     np.where((dfFinal[stock].isna()) & (dfFinal[i] == 1), 5,
                     np.where((dfFinal[stock].isna()) & (dfFinal[i] == 0), 6,
                     np.nan))))))


    # Map the numbers to the corresponding strings
    dfFinal[sinal] = dfFinal[sinal].map(mapping)
    

# 1: Há Stock e Sinal Positivo Ninja "Verdadeiramente Correcto"
# 2: Há Stock e Sinal Negativo Ninja "Falso Positivo"
# 3: Não há Stock e Sinal Positivo Ninja "Falso Negativo"
# 4: Não há Stock e Sinal Negativo Ninja "Rejeição Correcta"

> Ciclos

In [26]:
for i in produtos:
    ciclos = "CICLO %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    dfFinal[ciclos]=dfFinal[stock]/dfFinal[linear]

> Adequação

In [27]:
# Define the mappings of numbers to strings
mapping = {1: "Stock Suficiente",
           2: "Stock Insuf c Forn Adequado",
           3: "Stock Insuf c Forn Desadequado"}

for i in produtos:
    adequa = "ADEQUAÇÃO %s" % i
    ciclos = "CICLO %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    
    dfFinal[adequa] = np.where(dfFinal[ciclos] > 1.1, 1,
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] >= dfFinal[linear]), 2,
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] < dfFinal[linear]), 3, 
                      np.nan)))
    
    # Map the numbers to the corresponding strings
    dfFinal[adequa] = dfFinal[adequa].map(mapping)

- Dias para a rotura

In [29]:
for i in produtos:
    stock  = "STOCK %s" % i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" % i
    dias_rotura = "Dias_para_Rotura %s" % i
    
    dfFinal[dias_rotura] = dfFinal[stock]/dfFinal[sellout]

- Dias para a rotura de prateleira

In [40]:
for i in produtos:
    linear = "PRESLINEAR %s" % i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" % i
    dias_rotura_prat = "Dias_para_Rotura_Linear %s" % i
    
    dfFinal[dias_rotura_prat] = dfFinal[linear] / dfFinal[sellout]

In [41]:
dfFinal.head(20)

,DATA,STORE,Altura_do_Dia,Loja,Hora,Dia,Semana,CAFÉ DELTA Q MYTHIQ 80CAP,CAFÉ DELTA Q QALIDUS 80CAP,CAFÉ DELTA Q QALIDUS 10CAP,CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,CAFÉ BELLISSIMO INTENSO 200GR,CAFÉ DELTA RITUAL MU 220G,CAFÉ SOLÚVEL DELTA FRASCO 100G,CAFÉ SOLÚVEL DELTA FRASCO 200G,CEVADA SOLÚVEL DELTA FRASCO 200G,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,STOCK CAFÉ DELTA Q MYTHIQ 80CAP,PRESLINEAR CAFÉ DELTA Q MYTHIQ 80CAP,INTRANSIT CAFÉ DELTA Q MYTHIQ 80CAP,EXPECTED CAFÉ DELTA Q MYTHIQ 80CAP,SELLOUT CAFÉ DELTA Q MYTHIQ 80CAP,STOCK CAFÉ DELTA Q QALIDUS 80CAP,PRESLINEAR CAFÉ DELTA Q QALIDUS 80CAP,INTRANSIT CAFÉ DELTA Q QALIDUS 80CAP,EXPECTED CAFÉ DELTA Q QALIDUS 80CAP,SELLOUT CAFÉ DELTA Q QALIDUS 80CAP,STOCK CAFÉ DELTA Q QALIDUS 10CAP,PRESLINEAR CAFÉ DELTA Q QALIDUS 10CAP,INTRANSIT CAFÉ DELTA Q QALIDUS 10CAP,EXPECTED CAFÉ DELTA Q QALIDUS 10CAP,SELLOUT CAFÉ DELTA Q QALIDUS 10CAP,STOCK CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,PRESLINEAR CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,INTRANSIT CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,EXPECTED CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,SELLOUT CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,STOCK CAFÉ BELLISSIMO INTENSO 200GR,PRESLINEAR CAFÉ BELLISSIMO INTENSO 200GR,INTRANSIT CAFÉ BELLISSIMO INTENSO 200GR,EXPECTED CAFÉ BELLISSIMO INTENSO 200GR,SELLOUT CAFÉ BELLISSIMO INTENSO 200GR,STOCK CAFÉ DELTA RITUAL MU 220G,PRESLINEAR CAFÉ DELTA RITUAL MU 220G,INTRANSIT CAFÉ DELTA RITUAL MU 220G,EXPECTED CAFÉ DELTA RITUAL MU 220G,SELLOUT CAFÉ DELTA RITUAL MU 220G,STOCK CAFÉ SOLÚVEL DELTA FRASCO 100G,PRESLINEAR CAFÉ SOLÚVEL DELTA FRASCO 100G,INTRANSIT CAFÉ SOLÚVEL DELTA FRASCO 100G,EXPECTED CAFÉ SOLÚVEL DELTA FRASCO 100G,SELLOUT CAFÉ SOLÚVEL DELTA FRASCO 100G,STOCK CAFÉ SOLÚVEL DELTA FRASCO 200G,PRESLINEAR CAFÉ SOLÚVEL DELTA FRASCO 200G,INTRANSIT CAFÉ SOLÚVEL DELTA FRASCO 200G,EXPECTED CAFÉ SOLÚVEL DELTA FRASCO 200G,SELLOUT CAFÉ SOLÚVEL DELTA FRASCO 200G,STOCK CEVADA SOLÚVEL DELTA FRASCO 200G,PRESLINEAR CEVADA SOLÚVEL DELTA FRASCO 200G,INTRANSIT CEVADA SOLÚVEL DELTA FRASCO 200G,EXPECTED CEVADA SOLÚVEL DELTA FRASCO 200G,SELLOUT CEVADA SOLÚVEL DELTA FRASCO 200G,STOCK BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,PRESLINEAR BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,INTRANSIT BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,EXPECTED BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,SELLOUT BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Num_Produtos,ROTURA CAFÉ DELTA Q MYTHIQ 80CAP,ROTURA CAFÉ DELTA Q QALIDUS 80CAP,ROTURA CAFÉ DELTA Q QALIDUS 10CAP,ROTURA CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,ROTURA CAFÉ BELLISSIMO INTENSO 200GR,ROTURA CAFÉ DELTA RITUAL MU 220G,ROTURA CAFÉ SOLÚVEL DELTA FRASCO 100G,ROTURA CAFÉ SOLÚVEL DELTA FRASCO 200G,ROTURA CEVADA SOLÚVEL DELTA FRASCO 200G,ROTURA BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,PRÉ-ROTURA CAFÉ DELTA Q MYTHIQ 80CAP,PRÉ-ROTURA CAFÉ DELTA Q QALIDUS 80CAP,PRÉ-ROTURA CAFÉ DELTA Q QALIDUS 10CAP,PRÉ-ROTURA CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,PRÉ-ROTURA CAFÉ BELLISSIMO INTENSO 200GR,PRÉ-ROTURA CAFÉ DELTA RITUAL MU 220G,PRÉ-ROTURA CAFÉ SOLÚVEL DELTA FRASCO 100G,PRÉ-ROTURA CAFÉ SOLÚVEL DELTA FRASCO 200G,PRÉ-ROTURA CEVADA SOLÚVEL DELTA FRASCO 200G,PRÉ-ROTURA BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,SINAL CAFÉ DELTA Q MYTHIQ 80CAP,SINAL CAFÉ DELTA Q QALIDUS 80CAP,SINAL CAFÉ DELTA Q QALIDUS 10CAP,SINAL CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,SINAL CAFÉ BELLISSIMO INTENSO 200GR,SINAL CAFÉ DELTA RITUAL MU 220G,SINAL CAFÉ SOLÚVEL DELTA FRASCO 100G,SINAL CAFÉ SOLÚVEL DELTA FRASCO 200G,SINAL CEVADA SOLÚVEL DELTA FRASCO 200G,SINAL BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,CICLO CAFÉ DELTA Q MYTHIQ 80CAP,CICLO CAFÉ DELTA Q QALIDUS 80CAP,CICLO CAFÉ DELTA Q QALIDUS 10CAP,CICLO CAFÉ MOÍDO MÁQUINA CHÁVENA DELTA 250G,CICLO CAFÉ BELLISSIMO INTENSO 200GR,CICLO CAFÉ DELTA RITUAL MU 220G,CICLO CAFÉ SOLÚVEL DELTA FRASCO 100G,CICLO CAFÉ SOLÚVEL DELTA FRASCO 200G,CICLO CEVADA SOLÚVEL DELTA FRASCO 200G,CICLO BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,ADEQUAÇÃO CAFÉ DELTA Q MYTHIQ 80CAP,ADEQUAÇÃO CAFÉ DELTA Q QALIDUS 80CAP,ADEQUAÇÃO CAFÉ DELTA Q QALIDUS 10CAP,ADEQUAÇÃO 

# Acabamos com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição
- Rotura
- Pré-Rotura
- Sinal 
- Ciclos
- Adequação de Stock
- Dias para a rotura normal
- Dias para a rotura móvel XXXXXXXXXXX
- Dias para a rotura de prateleira
- Balanço

# Escrever

In [34]:
escrever_excel(dfFinal, "OUT_2_DataFusion")
#dfFinalCorr.to_csv('ficheiro_NSS.csv', index=False)

---

# 2022

## Ler ficheiro

In [ ]:
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta_2022=pd.read_excel("D:\\B&N Dados\\Delta\\cPadrão\\2Delta2022Id.xlsx", sheet_name="Sheet1")

## Mergir

In [ ]:
# Dataframe mergido no fim
df_2022=dfNinjas.copy()


for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta_2022[dfDelta_2022["DESC_ARTIGO"]==coluna][["id", "Semana", "STORE", "SOH", "PRES_STOCK", "INTRANSIT", "EXPECTED", "Dia_Actual"]]   #Dados Ninjas, seleccionar colunas do café específico
    
    #Juntar dados de acordo com o dia e a loja
    df_2022=pd.merge(df_2022, dfStocks, how="left", on = ["id","STORE", "Semana"])  
    
    #Mudar nomes para ser adaptado a cada produto
    df_2022=df_2022.rename(columns={"SOH": "STOCK %s" % coluna, "PRES_STOCK":"PRESLINEAR %s" % coluna, "INTRANSIT":"INTRANSIT %s" % coluna, "EXPECTED":"EXPECTED %s" % coluna, "Dia_Actual":"SELLOUT %s" % coluna})                       #Nomear coluna nova
    

## Escrever

In [ ]:
escrever_excel(df_2022, "OUT2DataFusion2022")